### 1. 安装HuggingFace 并下载模型到本地

In [ ]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

In [78]:
!rm -rf ./LLM_qwen_model

In [79]:
from huggingface_hub import snapshot_download
from pathlib import Path
local_model_path = Path("./LLM_qwen_model")
local_model_path.mkdir(exist_ok=True)
model_name = "Qwen/Qwen-7B-Chat"
commit_hash = "d25b5f81c8d96e90b0a779544e0adb4632dc5643"

In [80]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

'LLM_qwen_model/models--Qwen--Qwen-7B-Chat/snapshots/d25b5f81c8d96e90b0a779544e0adb4632dc5643'

### 2. 把模型拷贝到S3为后续部署做准备

In [84]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [85]:
s3_model_prefix = "LLM-RAG/workshop/LLM_qwen_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_qwen_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_qwen_deploy_code
model_snapshot_path: LLM_qwen_model/models--Qwen--Qwen-7B-Chat/snapshots/d25b5f81c8d96e90b0a779544e0adb4632dc5643


In [86]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_qwen_model/models--Qwen--Qwen-7B-Chat/snapshots/d25b5f81c8d96e90b0a779544e0adb4632dc5643/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_model/.gitattributes
upload: LLM_qwen_model/models--Qwen--Qwen-7B-Chat/snapshots/d25b5f81c8d96e90b0a779544e0adb4632dc5643/NOTICE to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_model/NOTICE
upload: LLM_qwen_model/models--Qwen--Qwen-7B-Chat/snapshots/d25b5f81c8d96e90b0a779544e0adb4632dc5643/LICENSE to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_model/LICENSE
upload: LLM_qwen_model/models--Qwen--Qwen-7B-Chat/snapshots/d25b5f81c8d96e90b0a779544e0adb4632dc5643/examples/react_prompt.md to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_model/examples/react_prompt.md
upload: LLM_qwen_model/models--Qwen--Qwen-7B-Chat/snapshots/d25b5f81c8d96e90b0a779544e0adb4632dc5643/README.md to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_model/README.md


### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [87]:
# inference_image_uri = (
#     f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed",
    region=sess.boto_session.region_name,
    version="0.22.1"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

# print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118


In [88]:
!mkdir -p LLM_qwen_deploy_code

In [89]:
%%writefile LLM_qwen_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", trust_remote_code=True, fp16=True).eval()
    model.generation_config = GenerationConfig.from_pretrained(model_location, trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参
    return model, tokenizer


model = None
tokenizer = None
generator = None


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data["history"]
    model.generation_config.max_new_tokens = params.get('max_length',1024)
    model.generation_config.top_p = params.get('top_p',0.8)
    response, history = model.chat(tokenizer, input_sentences, history=history)
    
    result = {"outputs": response, "history" : history}
    return Output().add_as_json(result)

Overwriting LLM_qwen_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [90]:
%%writefile LLM_qwen_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_model/

Overwriting LLM_qwen_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [91]:
%%writefile LLM_qwen_deploy_code/requirements.txt
transformers==4.31.0
accelerate==0.20.3 
transformers_stream_generator==0.0.4
tiktoken
einops

Overwriting LLM_qwen_deploy_code/requirements.txt


In [92]:
!rm model.tar.gz
!cd LLM_qwen_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_qwen_deploy_code

LLM_qwen_deploy_code/
LLM_qwen_deploy_code/model.py
LLM_qwen_deploy_code/requirements.txt
LLM_qwen_deploy_code/serving.properties


In [93]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [94]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"qwen") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

qwen-2023-08-09-07-48-39-409
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/qwen-2023-08-09-07-48-39-409


In [95]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/qwen-2023-08-09-07-48-39-409-config',
 'ResponseMetadata': {'RequestId': 'b01c1a6e-2dbe-4667-8a26-dbc3f2131f70',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b01c1a6e-2dbe-4667-8a26-dbc3f2131f70',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Wed, 09 Aug 2023 07:48:39 GMT'},
  'RetryAttempts': 0}}

In [96]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/qwen-2023-08-09-07-48-39-409-endpoint


#### 持续检测模型部署进度

In [97]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/qwen-2023-08-09-07-48-39-409-endpoint
Status: InService


### 5. 模型测试

In [98]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 2048,
  "top_p": 0.8
}

CPU times: user 4.48 ms, sys: 0 ns, total: 4.48 ms
Wall time: 14 ms


In [99]:
prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
请问AWS Clean Rooms是多方都会收费吗？
"""
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "outputs":"AWS Clean Rooms 是一种由 AWS 提供的隐私保护技术，用于在多个参与方之间共享数据。在使用 AWS Clean Rooms 进行数据共享时，只有查询的接收方需要付费，而发起者和数据贡献者不需要付费。",\n  "history":[\n    [\n      "AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。\\n请问AWS Clean Rooms是多方都会收费吗？\\n",\n      "AWS Clean Rooms 是一种由 AWS 提供的隐私保护技术，用于在多个参与方之间共享数据。在使用 AWS Clean Rooms 进行数据共享时，只有查询的接收方需要付费，而发起者和数据贡献者不需要付费。"\n    ]\n  ]\n}'

In [100]:
prompts1 = """AWS Clean Rooms 的FAQ文档有提到:\nQuestion: AWS Clean Rooms的一个协作中可以有多少个参与方? \nAnswer: 每个协作AWS Clean Rooms最多支持五名参与方。\n\n
请问AWS Clean Rooms的一个协作中可以有多少个参与方?"""

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

'{\n  "outputs":"每个协作AWS Clean Rooms最多支持五名参与方。",\n  "history":[\n    [\n      "AWS Clean Rooms 的FAQ文档有提到:\\nQuestion: AWS Clean Rooms的一个协作中可以有多少个参与方? \\nAnswer: 每个协作AWS Clean Rooms最多支持五名参与方。\\n\\n\\n请问AWS Clean Rooms的一个协作中可以有多少个参与方?",\n      "每个协作AWS Clean Rooms最多支持五名参与方。"\n    ]\n  ]\n}'

In [ ]:
%%time
prompts = """请写一篇介绍云南大理的文章，大约500字"""
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

In [ ]:
%%time
prompts = """请写一篇介绍云南大理的文章，大约100字"""
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode('utf8')

#### 清除模型Endpoint和config

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name  {endpoint_name}

In [ ]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [ ]:
!aws sagemaker delete-model --model-name {model_name}